In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import DistanceMetric
from scipy.spatial.distance import mahalanobis
import os
import sys
from PIL import Image
sys.path.append('../models')
import ResNet as resnet
import EfficientNet as efficientnet
import ViT as vit

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
##########################################
######### Configure Metric Setting ####### 
##########################################
batch_size = 64
weight_path = './weights/baseline/dp-sgd/celebA_dpsgd_resnet18_100.pt'
model = torch.load(weight_path)
criterion = nn.CrossEntropyLoss()
dataset_path = "/media/data2/data/CelebAMask-HQ/CelebA-HQ-img/"
annotation_path = "/media/data2/data/CelebAMask-HQ/CelebAMask-HQ-attribute-anno.txt"
data = 'CelebA'
##########################################
##########################################

In [ ]:
class CelebA(Dataset):
    def __init__(self, root, annotation_path, transform=None):
        self.transform = transform
        self.root_dir = root
        self.classes = [
            "Female, Not smiling, Young", "Female, Not smiling, Old", 
            "Female, Smiling, Young", "Female, Smiling, Old", 
            "Male, Not smiling, Young", "Male, Not smiling, Old", 
            "Male, Smiling, Young", "Male, Smiling, Old",
        ]
        
        self.image_paths = []
        self.image_labels = []

        anno_file = open(annotation_path, "r")
        num = int(anno_file.readline())
    
        line = anno_file.readline()
        attributes = list(line[:-1].split(' '))
        
        sex_index = attributes.index('Male') + 2
        smile_index = attributes.index('Smiling') + 2
        young_index = attributes.index('Young') + 2
        for _ in range(num):
            line = anno_file.readline()
            record = list(line[:-1].split(' '))
            self.image_paths.append(record[0])
            if record[sex_index] == "1":
                if record[smile_index] == "1":
                    if record[young_index] == "1":
                        self.image_labels.append(6)
                    else:
                        self.image_labels.append(7)
                else:
                    if record[young_index] == "1":
                        self.image_labels.append(4)
                    else:
                        self.image_labels.append(5)
            else:
                if record[smile_index] == "1":
                    if record[young_index] == "1":
                        self.image_labels.append(2)
                    else:
                        self.image_labels.append(3)
                else:
                    if record[young_index] == "1":
                        self.image_labels.append(0)
                    else:
                        self.image_labels.append(1)
        anno_file.close()
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_file_path = os.path.join(self.root_dir, self.image_paths[index])
        image = Image.open(image_file_path)

        label = self.image_labels[index]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [ ]:
mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) 
if data == 'Cifar-10':
    mean, std = (0.491, 0.482, 0.447), (0.247, 0.243, 0.262)
elif data == 'CelebA':
    mean, std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

model.to(device)
model.eval()

dataset =  CelebA(root=dataset_path, annotation_path=annotation_path, transform=trans)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=False, )

classes = dataset.classes

In [ ]:
latent_vectors = [[] for _ in range(len(classes))]

model.eval()
with torch.no_grad():
    for data in dataloader:
        images, labels = data[0].to(device), data[1].to(device)
        features, _ = model(images)

        deep_features = features.cpu().numpy().tolist()
        actual = labels.cpu().numpy().tolist()
        
        for i in range(len(actual)):
            latent_vectors[actual[i]].append(list(deep_features[i]))

In [ ]:
print("#####################################")
print("### Calculating Cosine Similarity ###")
print("#####################################")

total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    cssm = cosine_similarity(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(cssm)
    print(f"class: {classes[i]}, max: {np.max(cssm):.4f}, min: {np.min(cssm):.4f}, mean: {np.mean(cssm):.4f}, var: {np.var(cssm):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))

In [ ]:
print("#####################################")
print("## Calculating Euclidean Distance ###")
print("#####################################")

dist = DistanceMetric.get_metric('euclidean')
total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    euclidean = dist.pairwise(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(euclidean)
    print(f"class: {classes[i]}, max: {np.max(euclidean):.4f}, min: {np.min(euclidean):.4f}, mean: {np.mean(euclidean):.4f}, var: {np.var(euclidean):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))

In [ ]:
print("#####################################")
print("## Calculating Manhattan Distance ###")
print("#####################################")

dist = DistanceMetric.get_metric('manhattan')
total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    manhattan = dist.pairwise(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(manhattan)
    print(f"class: {classes[i]}, max: {np.max(manhattan):.4f}, min: {np.min(manhattan):.4f}, mean: {np.mean(manhattan):.4f}, var: {np.var(manhattan):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))

In [ ]:
print("#####################################")
print("## Calculating Chebyshev Distance ###")
print("#####################################")

dist = DistanceMetric.get_metric('chebyshev')
total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    chebyshev = dist.pairwise(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(chebyshev)
    print(f"class: {classes[i]}, max: {np.max(chebyshev):.4f}, min: {np.min(chebyshev):.4f}, mean: {np.mean(chebyshev):.4f}, var: {np.var(chebyshev):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))